In [ ]:
import pandas as pd

# Lees het CSV-bestand in
df = pd.read_csv('pad_naar_je_csv.csv')

# Converteer categorische variabelen naar dummy variabelen
df = pd.get_dummies(df, columns=['Product_category', 'Quality'], drop_first=True)
